In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import subprocess


def get_git_root():
    """Get the root directory of the git repository"""
    try:
        git_root = (
            subprocess.check_output(
                ["git", "rev-parse", "--show-toplevel"], stderr=subprocess.DEVNULL
            )
            .strip()
            .decode("utf-8")
        )
        return git_root
    except (subprocess.CalledProcessError, FileNotFoundError):
        return None


# Change to git root
git_root = get_git_root()
if git_root:
    os.chdir(git_root)
    print(f"Changed to git root: {os.getcwd()}")
else:
    print("Not in a git repository or git not found")

Changed to git root: /Users/sherman/GitHub/CSE-6748


In [3]:
import pandas as pd

import src.features.illuminate as illuminate
import src.features.index as index
import src.features.milestone as milestone
import src.features.student as student
import src.features.targets as targets
from src.feasy.sparkle import Sparkle
from src.utils import (
    NestedData,
    create_student_features,
    get_feature_functions_from_module,
    unnest,
)

pd.set_option("display.max_columns", None)
pd.options.mode.copy_on_write = True

In [ ]:
student_profile = pd.read_pickle("data/cleaned/student_profile_dataset.pkl")
student_profile.head()

,mask_studentpersonkey,GradeLevel,CurrentEnrollment,RetainedFlag,EnrollmentReasonDesc,WithDrawalReasonDesc,SchoolYearNumberFall,SchoolYearNumberSpring,SchoolStatusDesc,SchoolDetailReportName,SchoolDetailLevelDesc,SchoolStatusDescNextYear,SchoolDetailReportNameNextYear,SchoolDetailLevelDescNextYear,ActiveGiftedStudentResultRecordFlag,grad,milestone,illuminate,sat,rit
0,100075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"NestedData(TestingDateId=[11391], MathScore=[3...",NaN
1,104981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"NestedData(TestingDateId=[11261], MathScore=[5...",NaN
2,107055,6,N,N,Transferred from another GA district,Year End,2023,2024,Open,Woodland Middle,Middle School,Open,Woodland Middle,Middle School,D,NaN,NestedData(SubjectDesc=['English Language Arts...,NestedData(responsedatevalue=[datetime.date(20...,NaN,NaN
3,107055,7,N,N,Continuing in same school,Transfer to Another Public GA School,2024,2025,Open,Woodland Middle,Middle School,NaN,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN
4,114271,12,N,Y,Continuing in same school,High School Graduation,2021,2022,Open,Johns Creek High,High School,NaN,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN


In [5]:
# fmt:off
def analyze_dataset(df):
    """Comprehensive analysis of the merged student dataset"""
    
    import itertools

    import pandas as pd
    from IPython.display import display

    with pd.option_context(
        'display.max_columns', None,
        'display.width', None,
        'display.max_colwidth', None,
        'display.expand_frame_repr', False
    ):
    
        print("COMPREHENSIVE STUDENT DATASET ANALYSIS")
        print("=" * 70)
        
        # Basic metrics
        total_records = len(df)
        unique_students = df["mask_studentpersonkey"].nunique()
        
        # Current students filter
        current_students_df = df[df["CurrentEnrollment"] == "Y"]
        total_current_records = len(current_students_df)
        unique_current_students = current_students_df["mask_studentpersonkey"].nunique()
        
        # Dataset Overview
        overview_data = {
            'Metric': [
                'Total Records',
                'Unique Students', 
                'Average Records per Student',
                'Current Student Records',
                'Unique Current Students',
                'Average Records per Current Student'
            ],
            'Value': [
                f"{total_records:,}",
                f"{unique_students:,}",
                f"{total_records/unique_students:.2f}",
                f"{total_current_records:,}",
                f"{unique_current_students:,}",
                f"{total_current_records/unique_current_students:.2f}" if unique_current_students > 0 else "N/A"
            ]
        }
        
        print("\nDATASET OVERVIEW:")
        display(pd.DataFrame(overview_data))
        
        # Enrollment Coverage
        records_with_enrollment = df[~df["GradeLevel"].isnull()]
        records_missing_enrollment = df[df["GradeLevel"].isnull()]
        current_records_with_enrollment = current_students_df[~current_students_df["GradeLevel"].isnull()]
        current_records_missing_enrollment = current_students_df[current_students_df["GradeLevel"].isnull()]
        
        enrollment_data = {
            'Metric': [
                'Records with Enrollment',
                'Records missing Enrollment', 
                'Students with Enrollment',
                'Students missing Enrollment'
            ],
            'All Students': [
                f"{len(records_with_enrollment):,} ({len(records_with_enrollment)/total_records*100:.1f}%)",
                f"{len(records_missing_enrollment):,} ({len(records_missing_enrollment)/total_records*100:.1f}%)",
                f"{records_with_enrollment['mask_studentpersonkey'].nunique():,} ({records_with_enrollment['mask_studentpersonkey'].nunique()/unique_students*100:.1f}%)",
                f"{records_missing_enrollment['mask_studentpersonkey'].nunique():,} ({records_missing_enrollment['mask_studentpersonkey'].nunique()/unique_students*100:.1f}%)"
            ],
            'Current Students': [
                f"{len(current_records_with_enrollment):,} ({len(current_records_with_enrollment)/total_current_records*100:.1f}%)" if total_current_records > 0 else "0 (0.0%)",
                f"{len(current_records_missing_enrollment):,} ({len(current_records_missing_enrollment)/total_current_records*100:.1f}%)" if total_current_records > 0 else "0 (0.0%)",
                f"{current_records_with_enrollment['mask_studentpersonkey'].nunique():,} ({current_records_with_enrollment['mask_studentpersonkey'].nunique()/unique_current_students*100:.1f}%)" if unique_current_students > 0 else "0 (0.0%)",
                f"{current_records_missing_enrollment['mask_studentpersonkey'].nunique():,} ({current_records_missing_enrollment['mask_studentpersonkey'].nunique()/unique_current_students*100:.1f}%)" if unique_current_students > 0 else "0 (0.0%)"
            ]
        }
        
        print("\nENROLLMENT DATA COVERAGE:")
        display(pd.DataFrame(enrollment_data))
        
        # Assessment Coverage
        assessments = {
            "Graduation Data": "grad",
            "GA Milestones": "milestone", 
            "Illuminate": "illuminate",
            "SAT Scores": "sat",
            "RIT Scores": "rit",
        }
        
        assessment_data = []
        for name, col in assessments.items():
            # All students
            records_with_data = df[~df[col].isnull()]
            record_count = len(records_with_data)
            student_count = records_with_data["mask_studentpersonkey"].nunique()
            
            # Current students
            current_records_with_data = current_students_df[~current_students_df[col].isnull()]
            current_record_count = len(current_records_with_data)
            current_student_count = current_records_with_data["mask_studentpersonkey"].nunique()
            
            assessment_data.append({
                'Assessment': name,
                'All Students - Records': f"{record_count:,} ({record_count/total_records*100:.1f}%)",
                'All Students - Students': f"{student_count:,} ({student_count/unique_students*100:.1f}%)",
                'Current Students - Records': f"{current_record_count:,} ({current_record_count/total_current_records*100:.1f}%)" if total_current_records > 0 else "0 (0.0%)",
                'Current Students - Students': f"{current_student_count:,} ({current_student_count/unique_current_students*100:.1f}%)" if unique_current_students > 0 else "0 (0.0%)"
            })
        
        print("\nASSESSMENT DATA COVERAGE:")
        display(pd.DataFrame(assessment_data))
        
        # SAT Combination Analysis (All Students Only)
        non_sat_assessments = {k: v for k, v in assessments.items() if v != "sat"}
        sat_combinations = []
        
        # SAT + each individual assessment
        for name, col in non_sat_assessments.items():
            sat_combinations.append((f"SAT + {name}", ["sat", col]))
        
        # SAT + pairs
        for combo in itertools.combinations(non_sat_assessments.items(), 2):
            names = [item[0] for item in combo]
            cols = [item[1] for item in combo]
            sat_combinations.append((f"SAT + {' + '.join(names)}", ["sat"] + cols))
        
        # SAT + triplets
        for combo in itertools.combinations(non_sat_assessments.items(), 3):
            names = [item[0] for item in combo]
            cols = [item[1] for item in combo]
            sat_combinations.append((f"SAT + {' + '.join(names)}", ["sat"] + cols))
        
        # SAT + all
        all_names = list(non_sat_assessments.keys())
        all_cols = list(non_sat_assessments.values())
        sat_combinations.append((f"SAT + {' + '.join(all_names)}", ["sat"] + all_cols))
        
        sat_combo_data = []
        for combo_name, combo_cols in sat_combinations:
            # All students only
            mask = df[combo_cols[0]].notna()
            for col in combo_cols[1:]:
                mask = mask & df[col].notna()
            students_with_combo = df[mask]["mask_studentpersonkey"].nunique()
            
            sat_combo_data.append({
                'Combination': combo_name,
                'All Students': f"{students_with_combo:,} ({students_with_combo/unique_students*100:.1f}%)"
            })
        
        print("\nSAT COMBINATION ANALYSIS:")
        display(pd.DataFrame(sat_combo_data))
        
        # NON-SAT Combination Analysis (Current Students Only)
        if unique_current_students > 0:
            non_sat_combinations = []
            
            # Individual assessments (excluding SAT)
            for name, col in non_sat_assessments.items():
                non_sat_combinations.append((name, [col]))
            
            # Pairs of non-SAT assessments
            for combo in itertools.combinations(non_sat_assessments.items(), 2):
                names = [item[0] for item in combo]
                cols = [item[1] for item in combo]
                combo_name = f"{' + '.join(names)}"
                non_sat_combinations.append((combo_name, cols))
            
            # Triplets of non-SAT assessments
            for combo in itertools.combinations(non_sat_assessments.items(), 3):
                names = [item[0] for item in combo]
                cols = [item[1] for item in combo]
                combo_name = f"{' + '.join(names)}"
                non_sat_combinations.append((combo_name, cols))
            
            # All non-SAT assessments
            all_non_sat_names = list(non_sat_assessments.keys())
            all_non_sat_cols = list(non_sat_assessments.values())
            non_sat_combinations.append((f"{' + '.join(all_non_sat_names)}", all_non_sat_cols))
            
            current_combo_data = []
            for combo_name, combo_cols in non_sat_combinations:
                # Current students only
                current_mask = current_students_df[combo_cols[0]].notna()
                for col in combo_cols[1:]:
                    current_mask = current_mask & current_students_df[col].notna()
                
                current_students_with_combo = current_students_df[current_mask]["mask_studentpersonkey"].nunique()
                
                current_combo_data.append({
                    'Combination': combo_name,
                    'Current Students': f"{current_students_with_combo:,} ({current_students_with_combo/unique_current_students*100:.1f}%)"
                })
            
            print("\nNON-SAT COMBINATION ANALYSIS (CURRENT STUDENTS):")
            display(pd.DataFrame(current_combo_data))
        else:
            print("\nNON-SAT COMBINATION ANALYSIS (CURRENT STUDENTS):")
            print("No current students found in dataset.")

# fmt: on

analyze_dataset(student_profile)

COMPREHENSIVE STUDENT DATASET ANALYSIS

DATASET OVERVIEW:


,Metric,Value
0,Total Records,"603,464"
1,Unique Students,"159,092"
2,Average Records per Student,3.79
3,Current Student Records,"87,376"
4,Unique Current Students,"87,376"
5,Average Records per Current Student,1.00



ENROLLMENT DATA COVERAGE:


,Metric,All Students,Current Students
0,Records with Enrollment,"596,488 (98.8%)","87,376 (100.0%)"
1,Records missing Enrollment,"6,976 (1.2%)",0 (0.0%)
2,Students with Enrollment,"158,414 (99.6%)","87,376 (100.0%)"
3,Students missing Enrollment,"6,927 (4.4%)",0 (0.0%)



ASSESSMENT DATA COVERAGE:


,Assessment,All Students - Records,All Students - Students,Current Students - Records,Current Students - Students
0,Graduation Data,"29,859 (4.9%)","29,316 (18.4%)","7,241 (8.3%)","7,241 (8.3%)"
1,GA Milestones,"175,291 (29.0%)","86,976 (54.7%)",629 (0.7%),629 (0.7%)
2,Illuminate,"280,369 (46.5%)","120,820 (75.9%)","70,369 (80.5%)","70,369 (80.5%)"
3,SAT Scores,"55,427 (9.2%)","21,137 (13.3%)","6,947 (8.0%)","6,947 (8.0%)"
4,RIT Scores,"268,379 (44.5%)","82,381 (51.8%)","48,048 (55.0%)","48,048 (55.0%)"



SAT COMBINATION ANALYSIS:


,Combination,All Students
0,SAT + Graduation Data,"6,922 (4.4%)"
1,SAT + GA Milestones,"15,504 (9.7%)"
2,SAT + Illuminate,"12,064 (7.6%)"
3,SAT + RIT Scores,"15,527 (9.8%)"
4,SAT + Graduation Data + GA Milestones,"5,732 (3.6%)"
5,SAT + Graduation Data + Illuminate,"3,606 (2.3%)"
6,SAT + Graduation Data + RIT Scores,"6,757 (4.2%)"
7,SAT + GA Milestones + Illuminate,"11,420 (7.2%)"
8,SAT + GA Milestones + RIT Scores,"14,673 (9.2%)"
9,SAT + Illuminate + RIT Scores,"11,557 (7.3%)"



NON-SAT COMBINATION ANALYSIS (CURRENT STUDENTS):


,Combination,Current Students
0,Graduation Data,"7,241 (8.3%)"
1,GA Milestones,629 (0.7%)
2,Illuminate,"70,369 (80.5%)"
3,RIT Scores,"48,048 (55.0%)"
4,Graduation Data + GA Milestones,88 (0.1%)
5,Graduation Data + Illuminate,"6,883 (7.9%)"
6,Graduation Data + RIT Scores,"3,766 (4.3%)"
7,GA Milestones + Illuminate,301 (0.3%)
8,GA Milestones + RIT Scores,460 (0.5%)
9,Illuminate + RIT Scores,"38,511 (44.1%)"


In [6]:
contains_enrolment = student_profile["GradeLevel"].notna()
contains_sat = student_profile["sat"].notna()
contains_illuminate = student_profile["illuminate"].notna()
contains_milestone = student_profile["milestone"].notna()
current_students = student_profile["CurrentEnrollment"] == "Y"

In [7]:
data = student_profile[
    contains_enrolment & contains_sat & contains_illuminate & contains_milestone
]

data.head()

,mask_studentpersonkey,GradeLevel,CurrentEnrollment,RetainedFlag,EnrollmentReasonDesc,WithDrawalReasonDesc,SchoolYearNumberFall,SchoolYearNumberSpring,SchoolStatusDesc,SchoolDetailReportName,SchoolDetailLevelDesc,SchoolStatusDescNextYear,SchoolDetailReportNameNextYear,SchoolDetailLevelDescNextYear,ActiveGiftedStudentResultRecordFlag,grad,milestone,illuminate,sat,rit
229,1390795,9,N,N,Transferred from another GA district,Year End,2022,2023,Open,North Springs High,High School,Open,North Springs High,High School,NaN,"NestedData(SubjectArea=['Math', 'ELA', 'Electi...","NestedData(SubjectDesc=['Algebra I', 'Biology'...",NestedData(responsedatevalue=[datetime.date(20...,"NestedData(TestingDateId=[12542], MathScore=[5...","NestedData(TestingDateId=[11698, 11701, 11844,..."
694,1395580,11,N,N,Continuing in same school,Year End,2023,2024,Open,Milton High,High School,Open,Milton High,High School,NaN,NaN,NestedData(SubjectDesc=['American Literature &...,NestedData(responsedatevalue=[datetime.date(20...,"NestedData(TestingDateId=[11989], MathScore=[6...","NestedData(TestingDateId=[11721, 11735, 12077,..."
1179,1406902,9,N,N,Transferred from private school,Year End,2022,2023,Open,Riverwood High,High School,Open,Riverwood High,High School,NaN,"NestedData(SubjectArea=['Social Studies', 'Mat...","NestedData(SubjectDesc=['Algebra I', 'Biology'...",NestedData(responsedatevalue=[datetime.date(20...,"NestedData(TestingDateId=[12542], MathScore=[5...","NestedData(TestingDateId=[11705, 11708, 12076,..."
1180,1406902,10,N,N,Continuing in same school,Year End,2023,2024,Open,Riverwood High,High School,Open,Riverwood High,High School,NaN,NaN,NestedData(SubjectDesc=['American Literature &...,NestedData(responsedatevalue=[datetime.date(20...,"NestedData(TestingDateId=[12542], MathScore=[5...","NestedData(TestingDateId=[11705, 11708, 12076,..."
1357,1408618,11,N,N,Transferred from another GA district,Year End,2022,2023,Open,Hapeville Charter Career Academy,High School,Open,Hapeville Charter Career Academy,High School,NaN,NaN,NestedData(SubjectDesc=['American Literature &...,NestedData(responsedatevalue=[datetime.date(20...,"NestedData(TestingDateId=[12134], MathScore=[4...","NestedData(TestingDateId=[11692, 11693, 11839,..."


In [8]:
index_feat_funcs = get_feature_functions_from_module(index)
student_feat_funcs = get_feature_functions_from_module(student)
illuminate_feat_funcs = get_feature_functions_from_module(illuminate)
milestone_feat_funcs = get_feature_functions_from_module(milestone)
targets_feat_funcs = get_feature_functions_from_module(targets)

features = (
    index_feat_funcs
    + student_feat_funcs
    + illuminate_feat_funcs
    + milestone_feat_funcs
    + targets_feat_funcs
)

In [9]:
sparkle = Sparkle(features).source(data, from_dataframe=True)
features = sparkle.to_pandas()
features.head()

,SchoolYearFall,mask_studentpersonkey,GradeLevel,is_gifted,grade_mode_numeric,improvement_first_to_last,is_on_sat_grade_level,last_percent_correct,low_mastery_rate_below_50,mastery_rate_above_80,math_improvement_first_to_last,math_low_mastery_rate_below_50,math_mastery_rate_above_80,math_mean_item_pct_correct,math_percent_extension,math_percent_reteach,math_percent_review_practice,math_slope_percent_correct_over_time,math_std_item_pct_correct,math_total_points_earned,math_total_points_possible,math_weighted_percent_correct,mean_item_percent_correct,mean_standard_percent_correct,mean_std_pct_ela,mean_std_pct_math,num_assessments,num_items,percent_extension,percent_reteach,percent_review_practice,slope_percent_correct_over_time,std_item_percent_correct,total_points_earned,total_points_possible,verbal_improvement_first_to_last,verbal_low_mastery_rate_below_50,verbal_mastery_rate_above_80,verbal_mean_item_pct_correct,verbal_percent_extension,verbal_percent_reteach,verbal_percent_review_practice,verbal_slope_percent_correct_over_time,verbal_std_item_pct_correct,verbal_total_points_earned,verbal_total_points_possible,verbal_weighted_percent_correct,weighted_overall_percent_correct,achievement_level_mean_ela,achievement_level_mean_math,lexile_mean_ela,lexile_mean_math,mean_achievement_level_all,mean_lexile_score_all,mean_scale_score_all,num_subjects_tested,num_test_days,num_unique_test_dates,pct_proficient_all,pct_proficient_ela,pct_proficient_math,scale_score_improvement_all,scale_score_improvement_ela,scale_score_improvement_math,scale_score_mean_ela,scale_score_mean_math,scale_score_slope_all,scale_score_slope_ela,scale_score_slope_math,scale_score_std_ela,scale_score_std_math,std_scale_score_all,sat_math_score,sat_verbal_score
0,2022,1390795,9,N,9,11.76,0,100.00,0.235955,0.730337,11.76,0.313433,0.656716,71.757463,0.656716,0.343284,0.000000,0.140152,22.851185,912.0,1295.0,0.704247,78.176966,79.138652,98.485000,72.786119,89,89,0.730337,0.269663,0.000000,2.298548e-01,22.821853,1256.0,1647.0,0.00,0.000000,0.954545,97.727273,0.954545,0.045455,0.000000,0.000000e+00,3.006536,344.0,352.0,0.977273,0.762599,0.0,3.0,0.0,0.0,3.0,0.0,577.5,2,1,1,1.0,0.0,1.0,3.0,0.0,0.0,0.0,576.0,3.0,0.0,0.0,0.0,0.0,1.5,560,550
1,2023,1395580,11,N,11,0.00,1,86.67,0.166667,0.666667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,86.670000,79.166667,79.166667,0.000000,6,6,0.666667,0.166667,0.166667,1.732371e-14,0.000000,78.0,90.0,0.00,0.166667,0.666667,86.670000,0.666667,0.166667,0.166667,1.732371e-14,0.000000,78.0,90.0,0.866667,0.866667,3.0,0.0,1615.0,0.0,3.0,1615.0,586.0,1,1,1,1.0,1.0,0.0,0.0,0.0,0.0,586.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,640,690
2,2022,1406902,9,N,9,-14.34,0,56.25,0.358209,0.626866,-14.34,0.358209,0.626866,73.136567,0.611940,0.373134,0.014925,-0.171908,9.304384,958.0,1295.0,0.739768,73.136567,70.099552,0.000000,70.099552,67,67,0.611940,0.373134,0.014925,-1.719084e-01,9.304384,958.0,1295.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.000000,0.739768,0.0,3.0,0.0,0.0,3.0,0.0,535.5,2,2,2,1.0,0.0,1.0,-3.0,0.0,0.0,0.0,537.0,-3.0,0.0,0.0,0.0,0.0,1.5,560,530
3,2023,1406902,10,N,10,-42.31,0,50.00,0.371429,0.571429,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,72.833429,70.000000,70.000000,0.000000,35,35,0.571429,0.371429,0.057143,-1.175916e+00,16.248802,380.0,523.0,-42.31,0.371429,0.571429,72.833429,0.571429,0.371429,0.057143,-1.175916e+00,16.248802,380.0,523.0,0.726577,0.726577,3.0,0.0,1405.0,0.0,3.0,1405.0,579.0,1,1,1,1.0,1.0,0.0,0.0,0.0,0.0,579.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,560,530
4,2022,1408618,11,N,11,20.00,1,93.33,0.166667,0.750000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,83.330000,79.583333,79.583333,0.000000,12,12,0.666667,0.166667,0.166667,2.517483e+00,10.000000,150.0,180.0,20.00,0.166667,0.750000,83.330000,0.666667,0.166667,0.166667,2.517483e+00,10.000000,

In [10]:
STUDENT = "mask_studentpersonkey"
YEAR = "SchoolYearFall"
TARGET_MATH = "sat_math_score"
TARGET_VERBAL = "sat_verbal_score"

In [11]:
X_index, X = create_student_features(
    features,
    student_col=STUDENT,
    year_col=YEAR,
    target_cols=[TARGET_MATH, TARGET_VERBAL],
)

Y = (
    features.groupby(STUDENT)[[TARGET_MATH, TARGET_VERBAL]]
    .last()
    .reset_index(drop=True)
)

In [ ]:
display(X_index.head())
display(X.head())
display(Y.head())

,mask_studentpersonkey
0,1390795
1,1395580
2,1406902
3,1408618
4,1408636


,GradeLevel_latest,is_gifted_latest,grade_mode_numeric_latest,improvement_first_to_last_latest,is_on_sat_grade_level_latest,last_percent_correct_latest,low_mastery_rate_below_50_latest,mastery_rate_above_80_latest,math_improvement_first_to_last_latest,math_low_mastery_rate_below_50_latest,math_mastery_rate_above_80_latest,math_mean_item_pct_correct_latest,math_percent_extension_latest,math_percent_reteach_latest,math_percent_review_practice_latest,math_slope_percent_correct_over_time_latest,math_std_item_pct_correct_latest,math_total_points_earned_latest,math_total_points_possible_latest,math_weighted_percent_correct_latest,mean_item_percent_correct_latest,mean_standard_percent_correct_latest,mean_std_pct_ela_latest,mean_std_pct_math_latest,num_assessments_latest,num_items_latest,percent_extension_latest,percent_reteach_latest,percent_review_practice_latest,slope_percent_correct_over_time_latest,std_item_percent_correct_latest,total_points_earned_latest,total_points_possible_latest,verbal_improvement_first_to_last_latest,verbal_low_mastery_rate_below_50_latest,verbal_mastery_rate_above_80_latest,verbal_mean_item_pct_correct_latest,verbal_percent_extension_latest,verbal_percent_reteach_latest,verbal_percent_review_practice_latest,verbal_slope_percent_correct_over_time_latest,verbal_std_item_pct_correct_latest,verbal_total_points_earned_latest,verbal_total_points_possible_latest,verbal_weighted_percent_correct_latest,weighted_overall_percent_correct_latest,achievement_level_mean_ela_latest,achievement_level_mean_math_latest,lexile_mean_ela_latest,lexile_mean_math_latest,mean_achievement_level_all_latest,mean_lexile_score_all_latest,mean_scale_score_all_latest,num_subjects_tested_latest,num_test_days_latest,num_unique_test_dates_latest,pct_proficient_all_latest,pct_proficient_ela_latest,pct_proficient_math_latest,scale_score_improvement_all_latest,scale_score_improvement_ela_latest,scale_score_improvement_math_latest,scale_score_mean_ela_latest,scale_score_mean_math_latest,scale_score_slope_all_latest,scale_score_slope_ela_latest,scale_score_slope_math_latest,scale_score_std_ela_latest,scale_score_std_math_latest,std_scale_score_all_latest,grade_mode_numeric_delta,improvement_first_to_last_delta,is_on_sat_grade_level_delta,last_percent_correct_delta,low_mastery_rate_below_50_delta,mastery_rate_above_80_delta,math_improvement_first_to_last_delta,math_low_mastery_rate_below_50_delta,math_mastery_rate_above_80_delta,math_mean_item_pct_correct_delta,math_percent_extension_delta,math_percent_reteach_delta,math_percent_review_practice_delta,math_slope_percent_correct_over_time_delta,math_std_item_pct_correct_delta,math_total_points_earned_delta,math_total_points_possible_delta,math_weighted_percent_correct_delta,mean_item_percent_correct_delta,mean_standard_percent_correct_delta,mean_std_pct_ela_delta,mean_std_pct_math_delta,num_assessments_delta,num_items_delta,percent_extension_delta,percent_reteach_delta,percent_review_practice_delta,slope_percent_correct_over_time_delta,std_item_percent_correct_delta,total_points_earned_delta,total_points_possible_delta,verbal_improvement_first_to_last_delta,verbal_low_mastery_rate_below_50_delta,verbal_mastery_rate_above_80_delta,verbal_mean_item_pct_correct_delta,verbal_percent_extension_delta,verbal_percent_reteach_delta,verbal_percent_review_practice_delta,verbal_slope_percent_correct_over_time_delta,verbal_std_item_pct_correct_delta,verbal_total_points_earned_delta,verbal_total_points_possible_delta,verbal_weighted_percent_correct_delta,weighted_overall_percent_correct_delta,achievement_level_mean_ela_delta,achievement_level_mean_math_delta,lexile_mean_ela_delta,lexile_mean_math_delta,mean_achievement_level_all_delta,mean_lexile_score_all_delta,mean_scale_score_all_delta,num_subjects_tested_delta,num_test_days_delta,num_unique_test_dates_delta,pct_proficient_all_delta,pct_proficient_ela_delta,pct_proficient_math_delta,scale_score_improvement_all_delta,scale_score_improvement_ela_delta,scale_sc

,sat_math_score,sat_verbal_score
0,560,550
1,640,690
2,560,530
3,460,550
4,510,500


In [13]:
X_index.to_parquet("data/cleaned/X_index.parquet")
X.to_parquet("data/cleaned/X.parquet")
Y.to_parquet("data/cleaned/Y.parquet")